In [141]:
import pandas as pd

import json
# data/Amazon Import - 01.xlsx - import .xlsx file 


In [124]:
# import chardet

# def find_encoding(fname):
#     r_file = open(fname, 'rb').read()
#     result = chardet.detect(r_file)
#     return result['encoding']

# encoding = find_encoding('data/All_Listings_Report_US.txt')
# print(encoding)

In [125]:

all_listings_mapping_NA = pd.read_csv('data/all_listings_mapping_NA.csv', dtype=str)
BS_export_df = pd.read_csv('data/STOCK-STATUS202407098.952568.TXT', sep='\t', encoding='ascii', skiprows=2, dtype=str)

print(all_listings_mapping_NA.columns)
print(BS_export_df.columns)

Index(['seller_sku', 'ASIN_US', 'status_US', 'fulfillment_US', 'ASIN_CA',
       'status_CA', 'fulfillment_CA', 'ASIN_MX', 'status_MX', 'fulfillment_MX',
       'BS_SKU', 'B_SKU', 'B_NZ_SKU'],
      dtype='object')
Index(['CODE', 'NAME', 'SIZE', 'Unnamed: 3', 'Unnamed: 4', 'O-H', 'COM', 'B-O',
       'P-O', 'AVAIL', 'PROD TYPE'],
      dtype='object')


# Tasks: 
1. create a mapping file for BS file 
2. check is NAME column consistently match with SIZE column 

In [146]:
def BS_file_to_QT_map(BS_export_df, use_prod_type=False):
    # Select only needed columns and rename them
    BS_export_df = BS_export_df[['CODE', 'NAME', 'AVAIL', 'PROD TYPE']].rename(
        columns={'CODE': 'SKU'}
    )
    
    # Strip whitespace from string columns
    BS_export_df = BS_export_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    
    # Clean 'PROD TYPE' by removing "     00"
    BS_export_df['PROD TYPE'] = BS_export_df['PROD TYPE'].str.replace(" 00", "", regex=False).str.strip()


    # Convert 'AVAIL' to numeric, replacing non-numeric values with NaN
    BS_export_df['AVAIL'] = BS_export_df['AVAIL'].astype(int)
    
    # Replace NaN values with 0
    BS_export_df['AVAIL'] = BS_export_df['AVAIL'].fillna(0)
    
    
    # Create base dictionary mapping SKU to AVAIL
    sku_avail_map = dict(zip(BS_export_df['SKU'], BS_export_df['AVAIL']))
    
    if use_prod_type:
            # Filter DataFrame for related products
            mask = (BS_export_df['PROD TYPE'] != '') & (BS_export_df['AVAIL'] > 0)
            related_df = BS_export_df.loc[mask, ['SKU', 'AVAIL', 'PROD TYPE']]
            
            # Drop rows where 'PROD TYPE' is NaN
            related_df = related_df.dropna(subset=['PROD TYPE'])

            # Create dictionary mapping PROD TYPE to AVAIL
            prod_type_avail_map = dict(zip(related_df['PROD TYPE'], related_df['AVAIL']))


        
            
            # Update sku_avail_map for items with zero availability
            skus_for_update = [sku for sku, avail in sku_avail_map.items() if avail <= 0]
            
            # filter skus for update in prod_type_avail_map
            skus_for_update = [sku for sku in skus_for_update if sku in prod_type_avail_map.keys()]

            for sku in skus_for_update:
                sku_avail_map[sku] = prod_type_avail_map[sku]

    return sku_avail_map


sku_avail_map = BS_file_to_QT_map(BS_export_df)



print(sku_avail_map)

# save to json
with open('results/sku_avail_map.json', 'w') as f:
    json.dump(sku_avail_map, f)

{'FH37516S': 1, '.DEFECT': 0, '.DELIVERY': 0, '.DISCOUNT': 0, '.PRICEERROR': 0, '.RETURNS': 0, '001BF': 493, '001CY': 0, '001GF': 199, '001OB': 40, '001PB': 1016, '002BS': 11, '009CB': 50, '018OG': 81, '02579-05': 16, '02579-14': 16, '02579-14S': 1, '02579-18': 60, '02579-18S': 1, '02579-20S': 1, '02579-40': 140, '02579-40S': 1, '05-131': 0, '05-216': 0, '05-236': 0, '05-321': 0, '05-552': 0, '050.006': 0, '051.019': 0, '051.038': 0, '051.041': 0, '051.042': 0, '051.045': 0, '077GM': 10, '07CLEARANCE': 0, '090313': 0, '090344': 0, '094362': 0, '095271': 0, '095318': 0, '095363': 0, '10004-01': 90, '10004-02': 134, '10004-02S': 1, '10004-05': 14, '10004-07': 34, '10004-08': 22, '10004-10': 272, '10004-14': 68, '10004-15': 142, '10004-15S': 1, '10004-18': 12, '10004-19': 28, '10004-25': 92, '10004-26': 70, '10004-30': 64, '10004-31': 114, '10004-37': 80, '10004-37S': 1, '10004-38': 2, '10004-39': 40, '10004-44': 74, '10004-47': 76, '10006-11': 26, '10006-14': 42, '10006-15': 82, '10006-1

In [127]:


# BS_export_df = BS_export_df[['CODE', 'NAME', 'SIZE', 'Unnamed: 3', 'AVAIL', 'PROD TYPE']].copy()
# BS_export_df.columns = ['SKU', 'NAME', 'SIZE1', 'SIZE2', 'AVAIL', 'PROD TYPE']
# #strip all columns
# BS_export_df = BS_export_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# BS_export_df['PROD TYPE'] = BS_export_df['PROD TYPE'].str.replace("     00", "")
# print(BS_export_df.head())

In [128]:
prod_type = BS_export_df['PROD TYPE'].to_list()

print(set(prod_type))


{'B48-29     00', 'B48-09     00', 'EA5256     00', '26535P     00', '921-27     00', 'EA3404     00', '926-29     00', 'EA2658     00', 'EA3804     00', 'B48-08     00', '9004603    00', 'EA3201     00', 'EA2647     00', 'EA2801     00', 'B48-24     00', 'EA3091     00', '93465-1    00', 'B48-21     00', '111-00     00', '93943-1    00', 'EA2664     00', '703539     00', 'EA5705     00', 'EA2657     00', 'EA5093     00', '111-18     00', 'EA2110     00', 'EA5201     00', 'EA2646     00', 'CC800BPM   00', 'EA5403     00', '926-19     00', 'B48-19     00', '703600     00', 'EA2750     00', 'EA5100     00', 'EA2652     00', 'EA5101     00', '9004593    00', '926-27     00', 'EA5204     00', '921-17     00', 'B48-27     00', '703522     00', 'EA5104     00', 'EA2703     00', 'EA2660     00', '9346-51    00', 'EA3664     00', 'EA6303     00', '522973     00', 'EA2662     00', 'EA2701     00', 'B48-18     00', 'EA3252     00', '921-30     00', 'EA0018     00', '111-19     00', nan, '111-27 

In [129]:
SKU_list = BS_export_df['SKU'].to_list()

# finde unique cheracters from all strings in SKU_list
# unique_chars = set(''.join(SKU_list))
# print(unique_chars)

# fulter sku list for sku what contsin "-"
SKU_list = [sku for sku in SKU_list if "-" in sku]

print(len(SKU_list))

# remuve "-" from SKUs 
SKU_list = [sku.replace("-", "") for sku in SKU_list]

#finde unique skus and not unique skus
unique_SKUs = set(SKU_list)
not_unique_SKUs = [sku for sku in SKU_list if SKU_list.count(sku) > 1]
print(len(unique_SKUs))
print(len(not_unique_SKUs))

print(not_unique_SKUs)






KeyError: 'SKU'

In [ ]:


# # задача. Проверить для каждого уникального NAME является ли SIZE1 одинаковым для этого NAME 

# unique_names = BS_export_df['NAME'].unique().tolist()
# # print(len(unique_names))
# result =[]
# column ="SIZE1"
# for name in unique_names:
#     df = BS_export_df[BS_export_df['NAME'] == name]
#     if not df[column].mode().empty:
#         most_common_value = df[column].mode()[0]
#         result.append([name, most_common_value])

# #save to csv 

# result_df = pd.DataFrame(result, columns=['NAME', column])
# result_df.to_csv(f'results/name-{column}.csv', index=False)
   

